# LangChain Expression Language (LCEL)

The **L**ang**C**hain **E**xpression **L**anguage (LCEL) abstracts key Python concepts into a streamlined format, facilitating a "minimalist" code layer for constructing chains of LangChain components. LCEL offers robust support for:

1. Rapid development of chains.
2. Advanced features like streaming, asynchronous processing, parallel execution, and more.
3. Seamless integration with LangSmith and LangServe.

In [45]:
# !pip install -qU \
#     langchain==0.0.345 \
#     anthropic==0.7.7 \
#     cohere==4.37 \
#     docarray==0.39.1

## LCEL Syntax

In [46]:

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_ollama import ChatOllama
from rich import print as pp

# Define a prompt for generating book recommendations
prompt = ChatPromptTemplate.from_template(
    "Suggest some books for the genre {genre}"
)

# Use a different model
model = ChatOllama(model="llama3.2")

output_parser = StrOutputParser()

IIn a standard LangChain setup, we would connect these components using an LLMChain:

In [47]:
from langchain.chains import LLMChain

# Create a chain for book recommendations
chain = LLMChain(
    prompt=prompt,
    llm=model,
    output_parser=output_parser
)

# Run the chain
out = chain.run(genre="Science Fiction")
print(out)

Here are some popular and highly-recommended science fiction books across various sub-genres:

**Classic Sci-Fi**

1. "Dune" by Frank Herbert - A seminal work of sci-fi that explores politics, ecology, and human evolution on a desert planet.
2. "Foundation" by Isaac Asimov - A groundbreaking novel that spans centuries, exploring the rise and fall of an interstellar empire.
3. "2001: A Space Odyssey" by Arthur C. Clarke - A thought-provoking novel that explores the intersection of humanity and technology.

**Space Opera**

1. "The Three-Body Problem" by Liu Cixin - A award-winning novel that explores the first contact between humans and an alien civilization.
2. "Old Man's War" by John Scalzi - A fun, action-packed novel that explores the human condition through space travel and colonization.
3. "A Closed and Common Orbit" by Becky Chambers - A character-driven novel that explores identity, community, and acceptance in a sci-fi universe.

**Dystopian Sci-Fi**

1. "The Handmaid's Tale" b

Using LCEL the format is different, rather than relying on `Chains` we simple chain together each component using the pipe operator `|`:

In [48]:
lcel_chain = prompt | model | output_parser

# Run the chain
out = lcel_chain.invoke({"genre": "Science Fiction"})
print(out)

Here are some popular and highly-recommended science fiction books across various sub-genres:

**Classic Science Fiction**

1. "Dune" by Frank Herbert - A seminal work of epic science fiction set in a desert planet where humans have colonized.
2. "Foundation" by Isaac Asimov - A classic novel that explores the concept of psychohistory and the fate of humanity.
3. "1984" by George Orwell - A dystopian novel that depicts a totalitarian future society.

**Space Opera**

1. "The Three-Body Problem" by Liu Cixin - A award-winning novel that explores the first contact between humans and an alien civilization.
2. "Old Man's War" by John Scalzi - A fun and action-packed novel about a group of seniors who join the military to fight in an intergalactic war.
3. "The Expanse" series by James S. A. Corey - A gritty and realistic series set in a future where humanity has colonized the solar system.

**Cyberpunk**

1. "Neuromancer" by William Gibson - A groundbreaking novel that explores the intersec

## How the Pipe Operator Works
To truly grasp LCEL, let's examine how the pipe operation functions. It takes output from the **right** and feeds it to the **left**—but since this isn't standard Python, how is it implemented? We can create our own version using simple functions.

We'll utilize the `__or__` method in Python class objects. When we combine two classes like `chain = class_a | class_b`, the Python interpreter checks for the presence of the `__or__` method in these classes. If it exists, the expression `|` is translated to `chain = class_a.__or__(class_b)`. 

This means both of the following patterns yield the same result:

```python
# Object approach
chain = class_a.__or__(class_b)
chain("some input")

# Pipe approach
chain = class_a | class_b
chain("some input")
```

With this understanding, we can create a `Runnable` class that takes a function and transforms it into a chainable function using the pipe operator `|`.



In [49]:
class Runnable:
    def __init__(self, func):
        self.func = func

    def __or__(self, other):
        def chained_func(*args, **kwargs):
            return other(self.func(*args, **kwargs))
        return Runnable(chained_func)

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

In [50]:

def subtract_three(x):
    return x - 3

def divide_by_two(x):
    return x / 2

subtract_three = Runnable(subtract_three)
divide_by_two = Runnable(divide_by_two)



Using `__or__` directly we get the correct answer, now let's try using the pipe operator `|` to chain them together:

In [51]:

chain = subtract_three | divide_by_two
print(chain(10))  # Output: 3.5

3.5


Using either method we get the same response, at it's core this is what LCEL is doing, but there is more.

## LCEL Deep Dive
Now that we understand what this syntax is doing under the hood, let's explore it within the context of LCEL and see a few of the additional methods that LangChain has provided to maximize flexibility when working with LCEL.

### Runnables

When working with LCEL we may find that we need to modify the structure or values being passed between components — for this we can use _runnables_. Let's try:

In [52]:
from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

# Create an embedding model
embedding = OllamaEmbeddings(model="snowflake-arctic-embed:33m")

# Create two vector stores with different texts
vecstore_a = InMemoryVectorStore.from_texts(
    ["Python is a popular programming language", "Python is used for web development"],
    embedding=embedding
)
vecstore_b = InMemoryVectorStore.from_texts(
    ["Python is great for data science", "Python is known for its simplicity"],
    embedding=embedding
)

### Creating retrievers 

In [53]:

from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)

retriever_a = vecstore_a.as_retriever()
retriever_b = vecstore_b.as_retriever()

prompt_str = """Answer the following questions about Python:

Context: {context}

Question: {question}

Answer: """

prompt = ChatPromptTemplate.from_template(prompt_str)

retriever = RunnableParallel(
    {'context': retriever_a, 'question': RunnablePassthrough()},
)

chain = retriever | prompt | model | output_parser



In [54]:
out = chain.invoke("What is Python used for?")
print(out)

Based on the context, Python is used for web development.


Here we have used `RunnableParallel` to create two parallel streams of information, one for `"context"` that is information fed in from `retriever_a`, and another for `"question"` which is the _passthrough_ information, ie the information that is passed through from our `chain.invoke("when was James born?")` call.

Using this information the chain is close to answering the question but it doesn't have enough information, it is missing the information that we have stored in `retriever_b`. Fortunately, we can have multiple parallel information streams using the `RunnableParallel` object.

In [55]:

prompt_str = """Answer the question below using the context:

Context:
{context_a}
{context_b}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(prompt_str)

retrieval = RunnableParallel(
    {
        "context_a": retriever_a, "context_b": retriever_b,
        "question": RunnablePassthrough()
    }
)

chain = retrieval | prompt | model | output_parser

In [56]:
ut = chain.invoke("Why is Python popular?")
print(out)


Based on the context, Python is used for web development.


## Runnable Lambdas
The `RunnableLambda` is a LangChain abstraction that allows us to turn Python functions into pipe-compatible function, similar to the `Runnable` class we created near the beginning of this notebook.

Let's try it out with our earlier `add_five` and `multiply_by_two` functions.

In [57]:
def calc_cube(x):
    return x ** 3

def calc_reciprocal(x):
    return 1 / x

In [58]:
from langchain_core.runnables import RunnableLambda

# Wrap the functions with RunnableLambda
calc_cube = RunnableLambda(calc_cube)
calc_reciprocal = RunnableLambda(calc_reciprocal)


As with our earlier `Runnable` abstraction, we can use `|` operators to chain together our `RunnableLambda` abstractions.

In [59]:
chain = calc_cube | calc_reciprocal
print(chain.invoke(2))  # Output: 0.1chain = calc_cube | calc_reciprocal

0.125


Unlike our `Runnable` abstraction, we cannot run the `RunnableLambda` chain by calling it directly, instead we must call `chain.invoke`:

In [60]:
chain.invoke(3)

0.037037037037037035

As before, we can see the same answer. Naturally we can feed custom functions into our chains using this approach. Let's try a short chain and see where we might want to insert a custom function:

In [61]:
rompt_str = """Tell me a fun fact about {animal}"""
prompt = ChatPromptTemplate.from_template(prompt_str)

chain = prompt | model | output_parser

In [62]:
pp(chain.invoke({"animal": "elephants"}))

KeyError: "Input to ChatPromptTemplate is missing variables {'context_b', 'question', 'context_a'}.  Expected: ['context_a', 'context_b', 'question'] Received: ['animal']\nNote: if you intended {context_b} to be part of the string and not a variable, please escape it with double curly braces like: '{{context_b}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [ ]:
pp(chain.invoke({"topic": "Artificial Intelligence"}))

KeyError: "Input to ChatPromptTemplate is missing variables {'context_b', 'question', 'context_a'}.  Expected: ['context_a', 'context_b', 'question'] Received: ['topic']\nNote: if you intended {context_b} to be part of the string and not a variable, please escape it with double curly braces like: '{{context_b}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

The returned text always includes the initial `"Here's a short fact about ...\n\n"` — let's add a function to split on double newlines `"\n\n"` and only return the fact itself.

In [ ]:
def extract_fact(x):
    if "\n\n" in x:
        return "\n".join(x.split("\n\n")[1:])
    else:
        return x
    
get_fact = RunnableLambda(extract_fact)

chain = prompt | model | output_parser | get_fact
pp(chain)

KeyError: "Input to ChatPromptTemplate is missing variables {'context_b', 'question', 'context_a'}.  Expected: ['context_a', 'context_b', 'question'] Received: ['animal']\nNote: if you intended {context_b} to be part of the string and not a variable, please escape it with double curly braces like: '{{context_b}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [37]:
pp(chain.invoke({"topic": "Artificial Intelligence"}))

The term "Artificial Intelligence" (AI) was first coined in 1956 by John McCarthy, who used it to describe the goal
of creating machines that could simulate human intelligence and think like humans.

In [38]:
pp(chain.invoke({"topic": "Artificial Intelligence"}))

Did you know that the first AI program, called "Logical Theorist," was developed in 1956 by Allen Newell and 
Herbert Simon? This groundbreaking program could reason and solve problems using logical rules, marking the 
beginning of the field of Artificial Intelligence as we know it today.

Now we're getting well formatted outputs using our final custom `get_fact` function.

---

That covers the essentials you need to getting started and building with the **L**ang**C**hain **E**xpression **L**anguage (LCEL). With it, we can put together chains very easily — and the current focus of the LangChain team is on further LCEL development and support.

The pros and cons of LCEL are varied. Those that love it tend to focus on the minimalist code style, LCEL's support for streaming, parallel operations, and async, and LCEL's nice integration with LangChain's focus on chaining components together.

There are also people that are less fond of LCEL. Typically people will point to it being yet another abstraction on top of an already very abstract library, that the syntax is confusing, against [the Zen of Python](https://peps.python.org/pep-0020/), and requires too much effort to learn new (or uncommon) syntax.

Both viewpoints are entirely valid, LCEL is a very different approach — ofcourse there are major pros and major cons. In either case, if you're willing to spend some time learning the syntax, it allows us to develop very quickly, and with that in mind it's well worth learning.

---